### TODO:
1. Manually change "-ku" to actual city
2. Osaka vs Ōsaka
3. Change all prefecture 13 city to Tokyo
4. Some city are represented by List
5. Kyoto vs Kyōto
6. `"city": ".*Ō.*"`
7. Ōdate, 

In [1]:
import json
# from fuzzywuzzy import fuzz
import requests
from requests.exceptions import HTTPError
import time

In [2]:
stationsF = open("translated_stations_test.json")
stations = json.load(stationsF)

In [3]:
def normalize_long_vowels(text: str):
    return text.replace("Ō", "O").replace("ō", "o").replace("Ū", "U").replace("ū", "u")

In [ ]:
for s in stations:
    if isinstance(s['city'], list):
        if len(s['city']) == 1:
            # If there's only one item in the list, assign it as the city name
            s['city'] = normalize_long_vowels(s['city'][0])
        elif len(s['city']) > 1:
            # If there are multiple items in the list, set 'city' to None
            s['city'] = None
    if isinstance(s['city'], str):
        s['city'] = normalize_long_vowels(s['city'])
    # Extreme special case
    if s['city'] == 'Koda':
        s['city'] = 'Koda'
    if s['city'] == 'Nan\'yo':
        s['city'] = 'Nanyo'


In [23]:
with open("translated_stations_normalized.json", "w", encoding="utf-8") as of:
    json.dump(stations, of, ensure_ascii=False, indent=4)

### Add prefectures info to shinkansen

DONE: There are stations with same name in different prefectures. This leads to error in the "translated_stations_normalized.json" data. Need to redo the addition. (Manually modified JSON)

In [24]:
stations_to_add = []
s1 = ["東京", "品川", "新横浜", "小田原", "熱海", "三島", "新富士", "静岡", "掛川", "浜松", "豊橋", "三河安城", "名古屋", "岐阜羽島", "米原", "京都", "新大阪"]
stations_to_add.append(s1)
s2 = ["新大阪", "新神戸", "西明石", "姫路", "相生", "岡山", "新倉敷", "福山", "新尾道", "三原", "東広島", "広島", "新岩国", "徳山", "新山口", "厚狭", "新下関", "小倉", "博多"]
stations_to_add.append(s2)
s3 = ["東京", "上野", "大宮", "小山", "宇都宮", "那須塩原", "新白河", "郡山", "福島", "白石蔵王", "仙台", "古川", "くりこま高原", "一ノ関", "水沢江刺", "北上", "新花巻", "盛岡", "いわて沼宮内", "二戸", "八戸", "七戸十和田", "新青森"]
stations_to_add.append(s3)
s4 = ["東京", "上野", "大宮", "熊谷", "本庄早稲田", "高崎", "上毛高原", "越後湯沢", "浦佐", "長岡", "燕三条", "新潟"]
stations_to_add.append(s4)
s5 = []
# s5 = ["越後湯沢", "ガーラ湯沢"]
stations_to_add.append(s5)
s6 = ["福島", "米沢", "高畠", "赤湯", "かみのやま温泉", "山形", "天童", "さくらんぼ東根", "村山", "大石田", "新庄"]
stations_to_add.append(s6)
s7 = ["東京", "上野", "大宮", "小山", "宇都宮", "那須塩原", "新白河", "郡山", "福島", "白石蔵王", "仙台", "古川", "くりこま高原", "一ノ関", "水沢江刺", "北上", "新花巻", "盛岡", "雫石", "田沢湖", "角館", "大曲", "秋田"]
stations_to_add.append(s7)
s8 = ["東京", "上野", "大宮", "熊谷", "本庄早稲田", "高崎", "安中榛名", "軽井沢", "佐久平", "上田", "長野", "飯山", "上越妙高", "糸魚川", "黒部宇奈月温泉", "富山", "新高岡", "金沢", "小松", "加賀温泉", "芦原温泉", "福井", "越前たけふ", "敦賀", "東小浜", "京都", "松井山手", "新大阪"]
stations_to_add.append(s8)
s9 = ["博多", "新鳥栖", "久留米", "筑後船小屋", "新大牟田", "新玉名", "熊本", "新八代", "新水俣", "出水", "川内", "鹿児島中央"]
stations_to_add.append(s9)
s10 = ["新青森", "奥津軽いまべつ", "木古内", "新函館北斗"]
stations_to_add.append(s10)

In [55]:
def add_shinkansen_prefectures(stations: list[str], line: dict, stations_data_json: list[dict]):
    line_name = line["name_kanji"]
    line_id = line["ekidata_id"]
    prefectures = set()
    if stations is None:
        return

    for i, st in enumerate(stations):
        found = False
        for station_data in stations_data_json:
            # Check if station name matches any entry in station_data
            if station_data.get("name_kanji") == st or st in station_data.get("alternative_names", []):
                if found:
                    print(f"Station {st} was processed twice for line {line_name}, id {line_id}")
                    # break
                    continue
                found = True
                if line_id not in station_data["ekidata_line_ids"]:
                    print(f"Station {st} in prefecture {station_data['prefecture']} is passed")
                    continue
                # Get details for the existing station entry
                cur_prefecture = station_data["prefecture"]
                # print(f"For line {line_name}, station {st} is found in prefecture {cur_prefecture}")
                prefectures.add(cur_prefecture)
                line["prefectures"] = list(prefectures)
                
                # Break after finding and processing the correct station
                # break

        if not found:
            print(f"Station {st} is not found for line {line_name}")

In [57]:
for i in range(10):
    add_shinkansen_prefectures(stations_to_add[i], lines[i], stations)

Station 新富士 in prefecture 01 is passed
Station 岐阜羽島 is not found for line 東海道新幹線
Station 新尾道 is not found for line 山陽新幹線
Station 東広島 is not found for line 山陽新幹線
Station 新岩国 is not found for line 山陽新幹線
Station 郡山 was processed twice for line 東北新幹線, id 1004
Station 福島 was processed twice for line 東北新幹線, id 1004
Station 白石蔵王 is not found for line 東北新幹線
Station くりこま高原 is not found for line 東北新幹線
Station 水沢江刺 is not found for line 東北新幹線
Station 七戸十和田 is not found for line 東北新幹線
Station 本庄早稲田 is not found for line 上越新幹線
Station 上毛高原 is not found for line 上越新幹線
Station 福島 was processed twice for line 山形新幹線, id 1007
Station 郡山 was processed twice for line 秋田新幹線, id 1008
Station 福島 was processed twice for line 秋田新幹線, id 1008
Station 白石蔵王 is not found for line 秋田新幹線
Station くりこま高原 is not found for line 秋田新幹線
Station 水沢江刺 is not found for line 秋田新幹線
Station 本庄早稲田 is not found for line 北陸新幹線
Station 安中榛名 is not found for line 北陸新幹線
Station 黒部宇奈月温泉 is not found for line 北陸新幹線
Station 越前たけふ is not f

In [58]:
for i in range(10):
    print(json.dumps(lines[i], ensure_ascii=False, indent=4))

{
    "code": "",
    "ekidata_id": "1002",
    "name_kanji": "東海道新幹線",
    "name_kana": "とうかいどうしんかんせん",
    "name_romaji": "Tokaido Shinkansen",
    "alternative_names": [
        "トウカイドウシンカンセン"
    ],
    "prefectures": [
        "25",
        "22",
        "14",
        "27",
        "13",
        "26",
        "23"
    ],
    "logo": ""
}
{
    "code": "",
    "ekidata_id": "1003",
    "name_kanji": "山陽新幹線",
    "name_kana": "さんようしんかんせん",
    "name_romaji": "San'yo Shinkansen",
    "alternative_names": [
        "サンヨウシンカンセン"
    ],
    "prefectures": [
        "40",
        "28",
        "34",
        "35",
        "27",
        "33"
    ],
    "logo": ""
}
{
    "code": "",
    "ekidata_id": "1004",
    "name_kanji": "東北新幹線",
    "name_kana": "とうほくしんかんせん",
    "name_romaji": "Tohoku Shinkansen",
    "alternative_names": [
        "トウホクシンカンセン"
    ],
    "prefectures": [
        "02",
        "07",
        "13",
        "04",
        "03",
        "09",
        "11"
    ],
    "log

In [59]:
with open("processed_lines.json", "w", encoding="utf-8") as of:
    json.dump(lines, of, ensure_ascii=False, indent=4)

### Find "largest" station for each city

By "largest", we mean the station that can reach the most prefectures.

In [206]:
stationsF = open("translated_stations_normalized.json")
stations = json.load(stationsF)

In [207]:
linesF = open("processed_lines.json")
lines = json.load(linesF)

In [208]:
from collections import defaultdict
stations_by_city = defaultdict(list)  # (city: list(stations))

for s in stations:
    try:
        cur_city = s['city'] + "---" + s['prefecture']
        stations_by_city[cur_city].append(s)
    except:
        print(f'{s["name_kanji"]} is not processed because city name is not found')

幾寅 is not processed because city name is not found
落合 is not processed because city name is not found
月ケ岡 is not processed because city name is not found
豊ケ岡 is not processed because city name is not found
鹿折唐桑 is not processed because city name is not found
脇ノ沢 is not processed because city name is not found
小友 is not processed because city name is not found
細浦 is not processed because city name is not found
下船渡 is not processed because city name is not found
大船渡 is not processed because city name is not found


In [209]:
print(len(stations_by_city))

795


In [210]:
biggest_stations = {}  # city: (num_prefectures, list(station))
need_processing = []  # city names that require further attention

In [211]:
def get_prefectures(line_id):
    for line in lines:
        if line['ekidata_id'] == line_id:
            return line['prefectures']
    raise KeyError(f"{line_id} not found")

In [212]:
# Step 1: Calculate total reachable prefectures for each city
for ctys, stns in stations_by_city.items():
    all_reachable_prefectures = set()
    station_reach_count = {}  # {station_id: count of unique prefectures it can reach}
    all_stations_reach_one = True  # Flag to retain all if all reach only their own prefecture

    # Step 2: Track unique prefectures reachable by each station and accumulate all for the city
    for st in stns:
        lines_available = st['ekidata_line_ids']
        reachable = set()
        
        # Update reach of this station
        for line in lines_available:
            reachable.update(get_prefectures(line))
        
        # Track how many unique prefectures each station can reach
        uid = st['name_romaji'] + "---" + st['prefecture']
        station_reach_count[uid] = len(reachable)
        
        # Check if any station reaches more than its own prefecture
        if len(reachable) > 1:
            all_stations_reach_one = False
        
        # Accumulate unique reachable prefectures for the city
        all_reachable_prefectures.update(reachable)

    # Step 3: Check if all stations reach only one prefecture
    if all_stations_reach_one:
        # Retain all stations in `biggest_stations` for this city
        best_stations = stns
    else:
        # Select top 5 stations with the highest reach count and ensure uniqueness
        top_stations = sorted(station_reach_count.items(), key=lambda x: x[1], reverse=True)[:5]
        
        # Use a dictionary to store unique stations
        unique_best_stations = {}
        
        # Populate the unique stations dictionary
        for uid, _ in top_stations:
            for station in stns:
                station_uid = station['name_romaji'] + "---" + station['prefecture']
                if station_uid == uid:
                    unique_best_stations[station_uid] = station  # Retains only unique stations
        
        # Convert unique stations dictionary back to a list
        best_stations = list(unique_best_stations.values())

    # Step 4: Add city to `need_processing` if it only reaches one prefecture
    if len(all_reachable_prefectures) == 1:
        need_processing.append(ctys)
    
    # Assign results to `biggest_stations`
    biggest_stations[ctys] = (len(all_reachable_prefectures), best_stations)


In [213]:
print(len(biggest_stations["Noshiro---05"][1]))

5


In [214]:
print(len(need_processing))

98


In [215]:
from fuzzywuzzy import fuzz
# MATCH_THRESHOLD = 50
finished_processing = []

for cty in need_processing: 
    curr_selection = biggest_stations[cty][1]
    if len(curr_selection) == 1:
        finished_processing.append(cty)
        continue
    st_names = {st['name_romaji']: st for st in curr_selection}
    city_name = cty[:-5]  # Extract city name from cty
    
    # Check if any station matches the city name exactly
    if city_name in st_names.keys():
        biggest_stations[cty] = (1, [st_names[city_name]])
        finished_processing.append(cty)
    else:
        # Find stations where the city name appears in the station's romaji name
        partial_matches = [
            st for st in curr_selection 
            if fuzz.partial_ratio(city_name.lower(), st['name_romaji'].lower()) >= (len(city_name) - 2) / len(city_name) * 100
        ]
        
        if partial_matches:
            # Pick the first match or apply any additional ranking if needed
            biggest_stations[cty] = (1, partial_matches)
            finished_processing.append(cty)
        else:
            # If no partial match found, keep the original list and log a message
            biggest_stations[cty] = (1, [biggest_stations[cty][1][0]])
            finished_processing.append(cty)
            print(f"No station with name containing '{city_name}' found in {cty}, first station is picked")

No station with name containing 'Minamifurano' found in Minamifurano---01, first station is picked
No station with name containing 'Shinhidaka' found in Shinhidaka---01, first station is picked
No station with name containing 'Ozora' found in Ozora---01, first station is picked


In [11]:
# MATCH_THRESHOLD = 50
need_processing = []
# Process biggest_stations to handle multiple stations with the same max reachable prefectures
for cty, (num_prefectures, stations) in biggest_stations.items():
    # If there's only one best station, we keep it as is
    if len(stations) == 1:
        continue
    
    city_name = cty.split("---")[0]  # Extract the city name portion from the key
    # print(city_name.lower())
    # Find stations where the name closely matches the city name
    full_matches = [st for st in stations if city_name.lower() == st['name_romaji'].lower()]
    partial_matches = [
        st for st in stations
        if fuzz.partial_ratio(city_name.lower(), st['name_romaji'].lower()) >= (len(city_name) - 2) / len(city_name) * 100
    ]
    if full_matches:
        biggest_stations[cty] = (num_prefectures, [full_matches[0]])  # Keep only the first match
    # If we have at least one partial match, choose the first one (or apply a selection strategy)
    elif partial_matches:
        biggest_stations[cty] = (num_prefectures, [partial_matches[0]])  # Keep only the first match
    else:
        # If no fuzzy match is found, return the first one
        biggest_stations[cty] = (num_prefectures, [biggest_stations[cty][1][0]])
        print(f"No station with a name similar to '{city_name}' found in {cty}, first station in the list is picked")

No station with a name similar to 'Misato' found in Misato---05, first station in the list is picked
No station with a name similar to 'Kitaakita' found in Kitaakita---05, first station in the list is picked
No station with a name similar to 'Hirakawa' found in Hirakawa---02, first station in the list is picked
No station with a name similar to 'Happo' found in Happo---05, first station in the list is picked
No station with a name similar to 'Tsugaru' found in Tsugaru---02, first station in the list is picked
No station with a name similar to 'Nishiwaga' found in Nishiwaga---03, first station in the list is picked
No station with a name similar to 'Takizawa' found in Takizawa---03, first station in the list is picked
No station with a name similar to 'Semboku' found in Semboku---05, first station in the list is picked
No station with a name similar to 'Agano' found in Agano---15, first station in the list is picked
No station with a name similar to 'Tainai' found in Tainai---15, first 

In [216]:
cleaned_best_stations = {}

for city_key, (prefecture_count, stations) in biggest_stations.items():
    # Prepare a list to store the cleaned information for each station
    cleaned_stations = []

    for station in stations:
        # For lat/lon, use the first element in the `stations` field if it exists
        if station['stations']:
            station_info = station['stations'][0]
        else:
            station_info = {}

        # Gather the necessary fields for each station
        cleaned_station = {
            "name_kanji": station.get("name_kanji", ""),
            "name_romaji": station.get("name_romaji", ""),
            "city": station.get("city", ""),
            "prefecture": station.get("prefecture", ""),
            "ekidata_line_ids": station.get("ekidata_line_ids", []),
            # "lat": station_info.get("lat", None),
            # "lon": station_info.get("lon", None),
        }
        
        cleaned_stations.append(cleaned_station)

    # Create a new entry with the "reachable" prefecture count and the list of cleaned stations
    cleaned_best_stations[city_key] = {
        "reachable": prefecture_count,
        "stations": cleaned_stations
    }


In [217]:
with open("biggest_stations_big.json", "w", encoding="utf-8") as of:
    json.dump(cleaned_best_stations, of, ensure_ascii=False, indent=4)

Add station's ID corresponding to each line, add line names in case needed

In [218]:
stationsF = open("translated_stations_normalized.json")
stations = json.load(stationsF)

In [219]:
linesF = open("processed_lines.json")
lines = json.load(linesF)

In [220]:
bestStatF = open("biggest_stations_big.json")
best_stations = json.load(bestStatF)

In [221]:
prefF = open("open-data-jp-prefectures-master/prefectures.json")
prefectures = json.load(prefF)

In [222]:
prefecture_dict = {}

for prefecture in prefectures:
    iso_code = prefecture["iso"]
    # Split romaji and exclude the last word if more than one word
    romaji_parts = prefecture["prefecture_romaji"].split()
    if len(romaji_parts) > 1:
        romaji_name = " ".join(romaji_parts[:-1])
    else:
        romaji_name = romaji_parts[0]  # For single-word cases like "Hokkaido"
    
    # Store in dictionary
    prefecture_dict[iso_code] = romaji_name

# Output the result
print(prefecture_dict)

{'01': 'Hokkaido', '02': 'Aomori', '03': 'Iwate', '04': 'Miyagi', '05': 'Akita', '06': 'Yamagata', '07': 'Fukushima', '08': 'Ibaraki', '09': 'Tochigi', '10': 'Gunma', '11': 'Saitama', '12': 'Chiba', '13': 'Tokyo', '14': 'Kanagawa', '15': 'Niigata', '16': 'Toyama', '17': 'Ishikawa', '18': 'Fukui', '19': 'Yamanashi', '20': 'Nagano', '21': 'Gifu', '22': 'Shizuoka', '23': 'Aichi', '24': 'Mie', '25': 'Shiga', '26': 'Kyoto', '27': 'Osaka', '28': 'Hyōgo', '29': 'Nara', '30': 'Wakayama', '31': 'Tottori', '32': 'Shimane', '33': 'Okayama', '34': 'Hiroshima', '35': 'Yamaguchi', '36': 'Tokushima', '37': 'Kagawa', '38': 'Ehime', '39': 'Kōchi', '40': 'Fukuoka', '41': 'Saga', '42': 'Nagasaki', '43': 'Kumamoto', '44': 'Ōita', '45': 'Miyazaki', '46': 'Kagoshima', '47': 'Okinawa'}


In [223]:
def find_station(st_info):
    name = st_info['name_kanji']
    pref = st_info['prefecture']
    for st in stations:
        if st['name_kanji'] == name and st['prefecture'] == pref:
            return st['stations']
    print(f'station with {name} and in prefecture {pref} is not found')
    return None

def find_line(line_id):
    for l in lines:
        if l['ekidata_id'] == line_id:
            return l['name_kanji']
    return None

In [224]:
recorded_line_stations = defaultdict(list)

In [225]:
for city_pref, city_info in best_stations.items():
    # Iterate over each station in the list of stations for this city
    for station in city_info['stations']:
        stations_info = find_station(station)
        
        # Check if `stations_info` is None and handle accordingly
        if stations_info is None:
            print(f"Something strange happened with station {station['name_kanji']}")
            continue
        
        # Prepare lists to store `ekidata_line_ids` and `line_names` for the current station
        new_ids = []
        line_names = []
        
        # Process each related station info for current station
        for s_info in stations_info:
            # Generate new id format and record it
            new_id = s_info['ekidata_id'][:-2] + "---" + s_info['ekidata_id'][-2:]
            new_ids.append(new_id)
            
            # Update `recorded_line_stations` with current station data
            recorded_line_stations[int(s_info['ekidata_id'][:-2])].append(int(s_info['ekidata_id'][-2:]))
            
            # Find and store the line name for the current station
            line_name = find_line(s_info['ekidata_line_id'])
            line_names.append(line_name)
        
        # Assign `ekidata_line_ids` and `line_names` to the current station
        station['ekidata_line_ids'] = new_ids
        station['line_names'] = line_names


In [226]:
with open("biggest_stations_big.json", "w", encoding="utf-8") as of:
    json.dump(best_stations, of, ensure_ascii=False, indent=4)

In [238]:
bestStatF = open("biggest_stations_big.json")
best_stations = json.load(bestStatF)

In [239]:
for k, ls in recorded_line_stations.items():
    ls.sort()

In [240]:
def find_st_on_lineid(line, id):
    search = f"{line}---{id:02}"
    for city_pref, stat_info in best_stations.items():
        for st in stat_info['stations']:
            if search in st['ekidata_line_ids']:
                return city_pref, st
    print(f'Something wrong happened with {line}---{id:02}')
    return None, None

In [241]:
import pandas as pd

In [242]:
# df_routes = pd.DataFrame(columns=['from_city', 'to_city', 'type', 'from_station', 'to_station', 'distance', 'cost', 'route_id', 'enabled'])
df_routes_verbose = pd.DataFrame(columns=['from_city', 
                                          'from_region_id', 
                                          'from_region_name', 
                                          'to_city', 
                                          'to_region_id', 
                                          'to_region_name', 
                                          'type', 
                                          'from_station', 
                                          'to_station', 
                                          'duration', 
                                          'distance', 
                                          'cost', 
                                          'route_id', 
                                          'route_name', 
                                          'enabled'])

In [243]:
rows = []
for k, ls in recorded_line_stations.items():
    if len(ls) == 1:
        print(f"line {k} is not processed")
        continue
    for i in range(1, len(ls)):
        from_city, from_info = find_st_on_lineid(k, ls[i - 1])
        to_city, to_info = find_st_on_lineid(k, ls[i])
        rows.append({
            'from_city': from_city[:-5], 
            'from_region_id': int(from_city[-2:]) + 200, 
            'from_region_name': prefecture_dict[from_city[-2:]], 
            'to_city': to_city[:-5], 
            'to_region_id': int(to_city[-2:]) + 200, 
            'to_region_name': prefecture_dict[to_city[-2:]], 
            'type': 'train', 
            'from_station': from_info['name_romaji'], 
            'to_station': to_info['name_romaji'], 
            'duration': None,  # No available information
            'distance': None,  # No available information 
            'cost': None,  # No available information
            'route_id': f"JPRail {k}", 
            'route_name': find_line(str(k)), 
            'enabled': True
        })
        # Reverse edge
        rows.append({
            'to_city': from_city[:-5], 
            'to_region_id': int(from_city[-2:]) + 200, 
            'to_region_name': prefecture_dict[from_city[-2:]], 
            'from_city': to_city[:-5], 
            'from_region_id': int(to_city[-2:]) + 200, 
            'from_region_name': prefecture_dict[to_city[-2:]], 
            'type': 'train', 
            'to_station': from_info['name_romaji'], 
            'from_station': to_info['name_romaji'], 
            'duration': None,  # No available information
            'distance': None,  # No available information 
            'cost': None,  # No available information
            'route_id': f"JPRail {k}", 
            'route_name': find_line(str(k)), 
            'enabled': True
        })

In [244]:
df_routes_verbose = pd.concat([df_routes_verbose, pd.DataFrame(rows)], ignore_index=True)

In [245]:
print(df_routes_verbose.head(10).to_markdown())

|    | from_city   |   from_region_id | from_region_name   | to_city   |   to_region_id | to_region_name   | type   | from_station   | to_station   | duration   | distance   | cost   | route_id    | route_name   | enabled   |
|---:|:------------|-----------------:|:-------------------|:----------|---------------:|:-----------------|:-------|:---------------|:-------------|:-----------|:-----------|:-------|:------------|:-------------|:----------|
|  0 | Tokyo       |              213 | Tokyo              | Tokyo     |            213 | Tokyo            | train  | Tokyo          | Ueno         |            |            |        | JPRail 1009 | 北陸新幹線   | True      |
|  1 | Tokyo       |              213 | Tokyo              | Tokyo     |            213 | Tokyo            | train  | Ueno           | Tokyo        |            |            |        | JPRail 1009 | 北陸新幹線   | True      |
|  2 | Tokyo       |              213 | Tokyo              | Saitama   |            211 | Saitama         

In [246]:
df_routes_verbose.to_csv('processed_routes_big.csv', index=False)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('processed_routes_big.csv')
df[['from_city', 'from_region_name', 'to_city', 'to_region_name']] = df[['from_city', 'from_region_name', 'to_city', 'to_region_name']].apply(lambda x: x.str.lower())

In [3]:
df.to_csv('processed_routes_big.csv', index=False)

In [4]:
# Exclude specific columns from null check
subset_cols = [col for col in df.columns if col not in ['duration', 'distance', 'cost']]

# Count null rows in the subset
null_count = df[subset_cols].isnull().sum()

# Print the total number of null rows
print(f"Total null rows (excluding duration, distance, and cost): {null_count.sum()}")

Total null rows (excluding duration, distance, and cost): 0


In [5]:
import json

In [24]:
airportsF = open('Useful/processed_airports.json')
airports = json.load(airportsF)

In [25]:
prefF = open('Useful/prefectures.json')
prefs = json.load(prefF)

In [26]:
for ap in airports:
    ap['city'] = ap['city'].lower()
    ap['region_id'] = int("2" + ap['prefecture'])
    ap['prefecture'] = prefs[int(ap['prefecture']) - 1]['prefecture_romaji'].lower()
    ap['prefecture'] = ap['prefecture'].split(' ')[0]
    ap['region_name'] = ap.pop('prefecture')

In [27]:
with open("Useful/processed_airports.json", "w", encoding="utf-8") as of:
    json.dump(airports, of, ensure_ascii=False, indent=4)

In [10]:
str('Ō').lower()

'ō'

In [13]:
# Find rows with "ō" in either column
filtered_df = df[(df['from_region_name'].str.contains('ō')) | (df['to_region_name'].str.contains('ō'))]

print(filtered_df)

      from_city  from_region_id from_region_name    to_city  to_region_id  \
3044      osaka             227            osaka  amagasaki           228   
3045  amagasaki             228            hyōgo      osaka           227   
3046  amagasaki             228            hyōgo  amagasaki           228   
3047  amagasaki             228            hyōgo  amagasaki           228   
3048  amagasaki             228            hyōgo  amagasaki           228   
...         ...             ...              ...        ...           ...   
4551      saiki             244             ōita      saiki           244   
4552      saiki             244             ōita      saiki           244   
4553      saiki             244             ōita      saiki           244   
4554      saiki             244             ōita    nobeoka           245   
4555    nobeoka             245         miyazaki      saiki           244   

     to_region_name   type           from_station             to_station  \